In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os 
from datetime import datetime  # For timestamps
import time  # Additional import that might be useful for delays
import numpy as np
from jetbot import Robot, Camera



class LineDetector:
    def __init__(self, camera, width=224, height=224, save_dir='images_collection' ):
        self.camera = camera
        #self.camera = Camera.instance(width=width, height=height)
        self.image_height = height
        self.image_width = width
        
        self.save_dir = save_dir
        
        # Create directory for saving images if it doesn't exist
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
            os.makedirs(os.path.join(save_dir, 'original'))
            os.makedirs(os.path.join(save_dir, 'processed'))
            
    def preprocess_image(self, image):
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to reduce noise
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Apply threshold to get binary image
        _, binary = cv2.threshold(blurred, 111, 255, cv2.THRESH_BINARY_INV)

        
        roi_height = self.image_height // 2
        roi = binary[roi_height:, :]

        # Find connected components
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(roi, connectivity=8)

        # Get areas of all components (excluding background)
        areas = stats[1:, cv2.CC_STAT_AREA]  # Skip background (label 0)
        road = "small"
        if len(areas) > 0:
            # Calculate mean and median of component areas
            mean_area = np.mean(areas)
            median_area = np.median(areas)
            # Dynamically set minimum area threshold
            # If there are large components (like in the narrow road case),
            # set a higher threshold relative to the mean area
            if mean_area > 70:  # Adjust this threshold based on your cases
                print(f"small raod: {mean_area}")
                road = "small"
                min_area = mean_area * 0.6  # Remove components smaller than 10% of mean
                filtered_binary = np.zeros_like(roi)
                # Apply the adaptive threshold
                for label in range(1, num_labels):
                    area = stats[label, cv2.CC_STAT_AREA]
                    if area >= min_area:
                        #print(f"added part with area: {area} ")
                        filtered_binary[labels == label] = 255

                # Optional: Clean up the remaining components
                kernel = np.ones((3,3), np.uint8)
                filtered_binary = cv2.morphologyEx(filtered_binary, cv2.MORPH_CLOSE, kernel)
                filtered_binary = cv2.morphologyEx(filtered_binary, cv2.MORPH_OPEN, kernel)
            else:
                print(f"big raod: {mean_area}")
                road = "big"
                # For cases with smaller components (wider road),
                # use a more lenient threshold
                min_area = 5  # Minimum baseline threshold
                filtered_binary = roi  # If no components found, use original binary image           
        else:
            print("no components found")
            filtered_binary = roi  # If no components found, use original binary image

        return gray, blurred, filtered_binary, binary, road
        
        
    def detect_lines(self, binary_image):
        roi_height = 112
        roi = binary_image
        #roi = binary_image[roi_height:, :]

        edges = cv2.Canny(roi, 50, 150)

        lines = cv2.HoughLinesP(
            roi,
            rho=1,
            theta=np.pi/180,
            threshold=30,
            minLineLength=30,
            maxLineGap=80
        )

        left_lines = []
        right_lines = []

        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                if x2 - x1 == 0:
                    continue
                slope = (y2 - y1) / (x2 - x1)

                # Categorize lines as left or right based on slope and position
                line_center_x = (x1 + x2) / 2
                if slope < 0 and line_center_x < self.image_width/2:  # Left line
                    left_lines.append(line[0])
                elif slope > 0 and line_center_x > self.image_width/2:  # Right line
                    right_lines.append(line[0])

        return left_lines, right_lines, roi_height, roi
    
    def calculate_two_line_steering(self, left_lines, right_lines, roi_height):
        """
        Calculate steering angle based on position and angle between bottom lane points
        """
        image_center = self.image_width // 2

        # If no lines detected, go straight
        if not left_lines or not right_lines:
            return 0, image_center, image_center, 0, None

        # Find bottom points of each line
        def get_bottom_point(lines):
            max_y = -float('inf')
            bottom_point = None
            for line in lines:
                x1, y1, x2, y2 = line
                # Check both points of the line
                if y1 > max_y:
                    max_y = y1
                    bottom_point = (x1, y1)
                if y2 > max_y:
                    max_y = y2
                    bottom_point = (x2, y2)
            return bottom_point

        # Get bottom points
        left_bottom = get_bottom_point(left_lines)
        right_bottom = get_bottom_point(right_lines)

        if left_bottom is None or right_bottom is None:
            return 0, image_center, image_center, 0, None

        # Calculate position deviation
        left_x = left_bottom[0]
        right_x = right_bottom[0]
        center_x = (left_x + right_x) // 2
        deviation = center_x - image_center

        # Calculate angle between bottom points line and horizontal
        dx = right_bottom[0] - left_bottom[0]
        dy = right_bottom[1] - left_bottom[1]
        angle = np.arctan2(dy, dx) * 180 / np.pi

        # Combine position and angle for steering
        # Position component
        position_weight = 0.5
        angle_weight = 0.5

        max_angle = 30
        position_factor = deviation / (self.image_width / 2)  # Normalize to [-1, 1]
        angle_factor = angle / 45  # Normalize to [-1, 1], assuming max 45 degree deviation

        # Combine both factors
        steering_angle = -(position_weight * position_factor + angle_weight * angle_factor) * max_angle

        # Store bottom points for visualization
        bottom_points = {
            'left': (left_bottom[0], left_bottom[1] + roi_height),
            'right': (right_bottom[0], right_bottom[1] + roi_height),
            'angle': angle
        }

        return steering_angle, center_x, image_center, deviation, bottom_points

    def calculate_steering_angle(self, left_lines, right_lines, roi_height, road):
        """
        Calculate steering angle based on available lines
        """
        image_center = self.image_width // 2

        # Initialize previous_steering_angle if it doesn't exist
        if not hasattr(self, 'previous_steering_angle'):
            self.previous_steering_angle = 0

        # If no lines detected, maintain previous steering
        if not left_lines and not right_lines:
            return self.previous_steering_angle, image_center, image_center, 0, None

        # If both lines are detected, use the original two-line method
        if left_lines and right_lines:
            print("Mode 2 lines detected")
            # Get bottom points
            left_bottom = self.get_bottom_point(left_lines)
            right_bottom = self.get_bottom_point(right_lines)
            
            if left_bottom is None or right_bottom is None:
                
                return self.previous_steering_angle, image_center, image_center, 0, None
            
            # Calculate position deviation
            left_x = left_bottom[0]
            right_x = right_bottom[0]
            center_x = (left_x + right_x) // 2
            deviation = center_x - image_center
            
            # Calculate angle between bottom points
            dx = right_bottom[0] - left_bottom[0]
            dy = right_bottom[1] - left_bottom[1]
            angle = np.arctan2(dy, dx) * 180 / np.pi
            #print("test11113")
            # Combine position and angle for steering
            position_weight = 0.5
            angle_weight = 0.5
            max_angle = 30
            position_factor = deviation / (self.image_width / 2)
            angle_factor = angle / 45

            steering_angle = -(position_weight * position_factor + angle_weight * angle_factor) * max_angle
            #print("test11114")
            bottom_points = {
                'left': (left_bottom[0], left_bottom[1] + roi_height),
                'right': (right_bottom[0], right_bottom[1] + roi_height),
                'angle': angle,
                'single_line': False
            }
            bottom_point = (0, 0) 

        else:  # Single line case
            active_lines = left_lines if left_lines else right_lines
            using_right_line = bool(right_lines)

            # Get bottom and top points of the line
            bottom_point = self.get_bottom_point(active_lines)
            top_point = self.get_top_point(active_lines)

            if bottom_point is None or top_point is None:
                print("Mode noline previous steering")
                return self.previous_steering_angle, image_center, image_center, 0, None

            # Calculate angle relative to vertical
            dx = top_point[0] - bottom_point[0]
            dy = top_point[1] - bottom_point[1]
            angle = np.arctan2(dx, dy) * 180 / np.pi  # Angle relative to vertical

            # Calculate horizontal distance from image center to bottom point
            distance_to_center = abs(bottom_point[0] - image_center)
            max_distance = self.image_width / 2  # Maximum possible distance

            # Calculate vertical position factor
            #vertical_position = bottom_point[1]  # y-coordinate of bottom point
           
        
            max_vertical = self.image_height/4  # maximum y-value in ROI    
            #print(f"positions top 0: {top_point[0]:.2f}")
            #print(f"positions top 1: {top_point[1]:.2f}")
            #print(f"positions bottom 0: {bottom_point[0]:.2f}")
            #print(f"positions bottom 1: {bottom_point[1]:.2f}")
            vertical_position = (self.image_height/2) - bottom_point[1]  #distance from bottom to line in y
            vertical_factor = vertical_position / max_vertical  # 0 when at bottom, 1 when in the middle or higher
            distance_factor = 0
            # Calculate the target angle in steps:
            if using_right_line:
                print("Mode right line detected")
                if vertical_factor > 0.1:
                    # When line is not at bottom, prioritize vertical alignment
                    # Target goes from -100° (vertical_factor=1) to -145° (vertical_factor=0)
                    target_angle = -120 - (25 * (1 - vertical_factor))
                else:
                    # When line is at bottom, use horizontal distance
                    # Target goes from -145° (distance_factor=1) to -190° (distance_factor=0)
                    distance_factor = distance_to_center / max_distance
                    target_angle = -190 + (45 * distance_factor)
            else:
                print("Mode left line detected")
                if vertical_factor > 0.1:
                    # When line is not at bottom, prioritize vertical alignment
                    # Target goes from 100° (vertical_factor=1) to 145° (vertical_factor=0)
                    target_angle = 120 + (25 * (1 - vertical_factor))
                else:
                    # When line is at bottom, use horizontal distance
                    # Target goes from 145° (distance_factor=1) to 190° (distance_factor=0)
                    distance_factor = distance_to_center / max_distance
                    target_angle = 190 - (45 * distance_factor)

            #print(f"Distance to center: {distance_to_center:.2f}")
            #print(f"Vertical position: {vertical_position:.2f}")
            #print(f"Vertical factor: {vertical_factor:.2f}")
            #print(f"distance factor: {distance_factor:.2f}")
            print(f"Target angle: {target_angle:.1f}")

            steering_angle = -(angle - target_angle)
            steering_angle *= 0.5  # Proportional gain

            # Limit steering angle
            max_angle = 30
            steering_angle = np.clip(steering_angle, -max_angle, max_angle)

            # Calculate deviation for visualization
            deviation = bottom_point[0] - image_center

            bottom_points = {
                'single_line': True,
                'line_angle': angle,
                'bottom_point': (bottom_point[0], bottom_point[1] + roi_height),
                'top_point': (top_point[0], top_point[1] + roi_height),
                'using_right_line': using_right_line,
                'target_angle': target_angle,
                'distance_factor': distance_factor,
                'distance_to_center': distance_to_center,
                'vertical_factor': vertical_factor,
                'vertical_position': vertical_position
            }

        self.previous_steering_angle = steering_angle
        return steering_angle, bottom_point[0], image_center, deviation, bottom_points

    def get_bottom_point(self, lines):
        """Find the point with maximum y value (bottom-most point)"""
        max_y = -float('inf')
        bottom_point = None
        for line in lines:
            x1, y1, x2, y2 = line
            if y1 > max_y:
                max_y = y1
                bottom_point = (x1, y1)
            if y2 > max_y:
                max_y = y2
                bottom_point = (x2, y2)
        return bottom_point

    def get_top_point(self, lines):
        """Find the point with minimum y value (top-most point)"""
        min_y = float('inf')
        top_point = None
        for line in lines:
            x1, y1, x2, y2 = line
            if y1 < min_y:
                min_y = y1
                top_point = (x1, y1)
            if y2 < min_y:
                min_y = y2
                top_point = (x2, y2)
        return top_point

        
    def save_images(self, original, standard, probabilistic, timestamp, steering_angle):
        # Save original image
        cv2.imwrite(
            os.path.join(self.save_dir, 'original', f'frame_{timestamp}.jpg'),
            original
        )
        
        # Save processed images
        cv2.imwrite(
            os.path.join(self.save_dir, 'processed', f'frame_{timestamp}_standard.jpg'),
            standard
        )
        
        cv2.imwrite(
            os.path.join(self.save_dir, 'processed', f'frame_{timestamp}_probabilistic.jpg'),
            probabilistic
        )
        

        
    def process_frame(self):
        # Capture frame
        image = self.camera.value

        # Preprocess image
        gray, blurred, binary, binary_init, road = self.preprocess_image(image)

        # Detect lines
        left_lines, right_lines, roi_height, roi = self.detect_lines(binary)
        #print("test11")
        steering_angle, center_x, image_center, deviation, bottom_points = self.calculate_steering_angle(
        left_lines, right_lines, roi_height, road)
     

        # Visualize results
        visualization = image.copy()

        if left_lines or right_lines:
            # Draw all detected lines
            # Draw left lines in blue
            for line in left_lines:
                x1, y1, x2, y2 = line
                cv2.line(visualization, 
                        (x1, y1 + roi_height), 
                        (x2, y2 + roi_height), 
                        (255, 0, 0), 2)  # Blue color

            # Draw right lines in red
            for line in right_lines:
                x1, y1, x2, y2 = line
                cv2.line(visualization, 
                        (x1, y1 + roi_height), 
                        (x2, y2 + roi_height), 
                        (0, 0, 255), 2)  # Red color

            if bottom_points:
                bottom_y = self.image_height - 20  # Define bottom_y here for all cases

                if bottom_points.get('single_line', False):
                    # Single line visualization
                    # Draw bottom and top points
                    bottom = bottom_points['bottom_point']
                    top = bottom_points['top_point']

                    cv2.circle(visualization, bottom, 5, (255, 255, 0), -1)  # Yellow
                    cv2.circle(visualization, top, 5, (255, 255, 0), -1)  # Yellow

                    # Draw line between top and bottom points
                    cv2.line(visualization,
                            bottom,
                            top,
                            (0, 255, 255), 2)  # Yellow

                    # Draw vertical reference line
                    cv2.line(visualization,
                            (image_center, bottom[1]),
                            (image_center, top[1]),
                            (0, 255, 0), 1)  # Green line

                    # Add text for single line mode
                    cv2.putText(visualization,
                               f"Mode: {'Right' if bottom_points['using_right_line'] else 'Left'} Line Only",
                               (10, bottom_y - 85),
                               cv2.FONT_HERSHEY_SIMPLEX,
                               0.5,
                               (255, 255, 255),
                               1)
                    # Add visualization for adaptive target angle
                    if 'target_angle' in bottom_points:
                        # Draw current line angle
                        cv2.putText(visualization,
                                   f"Current Angle: {bottom_points['line_angle']:.1f}°",
                                   (10, bottom_y - 105),
                                   cv2.FONT_HERSHEY_SIMPLEX,
                                   0.5,
                                   (255, 255, 255),
                                   1)

                        # Draw target angle
                        cv2.putText(visualization,
                                   f"Target Angle: {bottom_points['target_angle']:.1f}°",
                                   (10, bottom_y - 125),
                                   cv2.FONT_HERSHEY_SIMPLEX,
                                   0.5,
                                   (255, 255, 255),
                                   1)

                        # In the visualization part:
                        cv2.putText(visualization,
                                   f"Dist to Center: {bottom_points['distance_to_center']:.0f}px",
                                   (10, bottom_y - 165),
                                   cv2.FONT_HERSHEY_SIMPLEX,
                                   0.5,
                                   (255, 255, 255),
                                   1)

                        cv2.putText(visualization,
                                   f"Vert Factor: {bottom_points['vertical_factor']:.2f}",
                                   (10, bottom_y - 145),
                                   cv2.FONT_HERSHEY_SIMPLEX,
                                   0.5,
                                   (255, 255, 255),
                                   1)

                        cv2.putText(visualization,
                                   f"Dist Factor: {bottom_points['distance_factor']:.2f}",
                                   (10, bottom_y - 125),
                                   cv2.FONT_HERSHEY_SIMPLEX,
                                   0.5,
                                   (255, 255, 255),
                                   1)
                        # Draw target angle line
                        arrow_length = 40
                        target_angle_rad = np.radians(bottom_points['target_angle'])
                        target_x = int(arrow_length * np.sin(target_angle_rad))
                        target_y = int(arrow_length * np.cos(target_angle_rad))

                        cv2.line(visualization,
                                (image_center, bottom_y - 60),
                                (image_center + target_x, bottom_y - 60 - target_y),
                                (0, 255, 0),  # Green color for target
                                1)
                else:
                    # Dual line visualization
                    cv2.circle(visualization, bottom_points['left'], 5, (255, 255, 0), -1)  # Yellow
                    cv2.circle(visualization, bottom_points['right'], 5, (255, 255, 0), -1)  # Yellow

                    # Draw line between bottom points
                    cv2.line(visualization,
                            bottom_points['left'],
                            bottom_points['right'],
                            (0, 255, 255), 2)  # Yellow

                    # Draw horizontal reference line
                    mid_y = (bottom_points['left'][1] + bottom_points['right'][1]) // 2
                    left_x = min(bottom_points['left'][0], bottom_points['right'][0])
                    right_x = max(bottom_points['left'][0], bottom_points['right'][0])
                    cv2.line(visualization,
                        (left_x, mid_y),
                        (right_x, mid_y),
                        (0, 255, 0), 1)  # Green line

                # Draw correction visualization (common for both modes)
                # Draw image center line
                cv2.line(visualization,
                        (image_center, bottom_y - 40),
                        (image_center, bottom_y),
                        (255, 255, 0), 2)

                # Draw detected center point
                cv2.line(visualization,
                        (center_x, bottom_y - 40),
                        (center_x, bottom_y),
                        (0, 255, 0), 2)

                # Draw deviation line
                cv2.line(visualization,
                        (image_center, bottom_y - 20),
                        (center_x, bottom_y - 20),
                        (255, 0, 255), 2)

                # Add text annotations
                cv2.putText(visualization,
                           f"Deviation: {deviation}px",
                           (10, bottom_y - 65),
                           cv2.FONT_HERSHEY_SIMPLEX,
                           0.5,
                           (255, 255, 255),
                           1)

                angle_value = bottom_points.get('line_angle' if bottom_points.get('single_line', False) else 'angle', 0)
                cv2.putText(visualization,
                           f"Angle: {angle_value:.1f} deg",
                           (10, bottom_y - 45),
                           cv2.FONT_HERSHEY_SIMPLEX,
                           0.5,
                           (255, 255, 255),
                           1)

                cv2.putText(visualization,
                           f"Steering: {steering_angle:.1f} deg",
                           (10, bottom_y - 25),
                           cv2.FONT_HERSHEY_SIMPLEX,
                           0.5,
                           (255, 255, 255),
                           1)

                # Draw steering direction arrow
                arrow_length = 40
                arrow_angle = np.radians(steering_angle)
                arrow_x = int(arrow_length * np.sin(arrow_angle))
                arrow_y = int(arrow_length * np.cos(arrow_angle))

                arrow_start = (image_center, bottom_y - 60)
                arrow_end = (image_center + arrow_x, bottom_y - 60 - arrow_y)

                cv2.arrowedLine(visualization,
                              arrow_start,
                              arrow_end,
                              (0, 255, 255),
                              2,
                              tipLength=0.3)

        return steering_angle, {
            'original': cv2.cvtColor(image, cv2.COLOR_BGR2RGB),
            'grayscale': gray,
            'blurred': blurred,
            'binary': binary_init,
            'roi': roi,
            'result': cv2.cvtColor(visualization, cv2.COLOR_BGR2RGB)
        }
    def stop(self):
        self.camera.stop()

def plot_processing_steps(images, steering_angle):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f'Line Detection Processing Steps (Steering Angle: {steering_angle:.2f}°)', fontsize=16)
    
    # Plot original image
    axes[0, 0].imshow(images['original'])
    axes[0, 0].set_title('Original Image')
    axes[0, 0].axis('off')
    
    # Plot grayscale image
    axes[0, 1].imshow(images['grayscale'], cmap='gray')
    axes[0, 1].set_title('Grayscale')
    axes[0, 1].axis('off')
    
    # Plot blurred image
    axes[0, 2].imshow(images['blurred'], cmap='gray')
    axes[0, 2].set_title('Gaussian Blur')
    axes[0, 2].axis('off')
    
    # Plot binary image
    axes[1, 0].imshow(images['binary'], cmap='gray')
    axes[1, 0].set_title('Binary Threshold')
    axes[1, 0].axis('off')
    
    # Plot ROI
    axes[1, 1].imshow(images['roi'], cmap='gray')
    axes[1, 1].set_title('Region of Interest')
    axes[1, 1].axis('off')
    
    # Plot final result with detected lines
    axes[1, 2].imshow(images['result'])
    axes[1, 2].set_title('Detected Lines')
    axes[1, 2].axis('off')
    
    plt.tight_layout()
    plt.show()



In [12]:

class PIDController:
    def __init__(self, kp, ki, kd):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        
        self.previous_error = 0
        self.integral = 0
        
    def compute(self, error, dt):
        """
        Compute PID control value based on error and time step
        """
        # Proportional term
        p_term = self.kp * error
        
        # Integral term
        self.integral += error * dt
        i_term = self.ki * self.integral
        
        # Derivative term
        derivative = (error - self.previous_error) / dt
        d_term = self.kd * derivative
        
        # Save error for next iteration
        self.previous_error = error
        
        # Compute total control value
        control = p_term + i_term + d_term
        
        return control
        
    def reset(self):
        """Reset the PID controller"""
        self.previous_error = 0
        self.integral = 0

In [15]:
 from IPython.display import clear_output
class LineFollower:
    def __init__(self, robot, camera, base_speed=0.3):
        # Initialize robot and camera
        #self.robot = Robot()
        #self.camera = Camera.instance(width=224, height=224)
        self.robot = robot
        self.camera = camera
        self.detector = LineDetector(camera = self.camera)
        
        # PID controller for steering
        # Adjust these values based on testing
        self.pid = PIDController(kp=0.03, ki=0.0008, kd=0.001)
        
        # Motor control parameters
        self.base_speed = base_speed
        self.max_steering = 0.2  # Maximum steering adjustment
        
        # Time tracking for PID
        self.previous_time = time.time()
        
    def steer(self, steering_value):
        """
        Convert steering value to motor commands
        steering_value should be between -1 and 1
        """
        # Clip steering value
        steering_value = np.clip(steering_value, -1, 1)
        
        # Calculate motor values
        if steering_value >= 0:
            # Turning right
            left_speed = self.base_speed
            right_speed = self.base_speed * (1 - steering_value)
        else:
            # Turning left
            left_speed = self.base_speed * (1 + steering_value)
            right_speed = self.base_speed
            
        print(f"Left speed: {left_speed}")
        print(f"Right speed: {right_speed}")
        # Apply motor values
        self.robot.left_motor.value = float(left_speed)
        self.robot.right_motor.value = float(right_speed)
        
    def follow_line(self):
        """Main line following loop"""
        try:
            while True:
                # Calculate time step
                current_time = time.time()
                dt = current_time - self.previous_time
                self.previous_time = current_time
                
                # Get steering angle from your existing line detection code
                steering_angle, images = self.detector.process_frame()
                #print("test1")
                #clear_output(wait=True)
                #plot_processing_steps(images, steering_angle)
                #steering_angle, visualization = self.process_frame()  # Your existing method
                print(f"steering angel: {steering_angle}")
                # Convert steering angle to normalized error (-1 to 1)
                error = steering_angle / 30.0  # Assuming max steering angle is 30 degrees
                #print(f"error : {error}")
                # Compute PID control value
                control_value = self.pid.compute(steering_angle, dt)
                #print(f"controller : {control_value}")
                # Apply steering
                self.steer(control_value)
                
                # Optional: Display visualization
                #cv2.imshow('Line Following', images['result'])
                time.sleep(0.3)
                
                #if cv2.waitKey(1) & 0xFF == ord('q'):
                 #   break
                
                
        except KeyboardInterrupt:
            print("Stopping robot...")
        finally:
            # Stop the robot
            self.robot.stop()
            cv2.destroyAllWindows()
            self.camera.stop()
    
    def stop(self):
        """Stop the robot"""
        self.robot.left_motor.value = 0.0
        self.robot.right_motor.value = 0.0

In [16]:
    robot = Robot()
    camera = Camera.instance(width=224, height=224)
    follower = LineFollower(robot, camera,base_speed=0.3)
   
    try:
        # Start line following
        follower.follow_line()
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Make sure robot stops
        follower.stop()
        camera.stop()
        robot.stop()

small raod: 1989.857142857143
Mode left line detected
Target angle: 145.0
steering angel: 6.168652929561901
Left speed: 0.4
Right speed: 0.0
small raod: 3854.0
Mode 2 lines detected
steering angel: -4.269866754236967
Left speed: 0.3381062854165528
Right speed: 0.4
small raod: 1541.8333333333333
Mode left line detected
Target angle: 146.2
steering angel: 2.620435020802006
Left speed: 0.4
Right speed: 0.36270662810522786
small raod: 979.3333333333334
Mode 2 lines detected
steering angel: 2.2088701748879638
Left speed: 0.4
Right speed: 0.3738109141135011
small raod: 2195.6666666666665
Mode left line detected
Target angle: 147.4
steering angel: 3.60814268898973
Left speed: 0.4
Right speed: 0.3546561839137615
small raod: 1758.3333333333333
Mode left line detected
Target angle: 149.0
steering angel: 4.004425592681301
Left speed: 0.4
Right speed: 0.35052248045164985
small raod: 750.25
Mode 2 lines detected
steering angel: 2.485207118541989
Left speed: 0.4
Right speed: 0.370624967651157
small 

In [47]:
camera.stop()

In [48]:
robot.stop()

In [ ]:
from logging import raiseExceptions
from cv2 import VideoCapture, imshow, rectangle, waitKey, destroyAllWindows, CAP_PROP_FRAME_WIDTH, CAP_PROP_FRAME_HEIGHT
from pyzbar.pyzbar import decode as decode_qr
import time

class QRCodeDetect:
    def __init__(self):
        """Initialize the camera"""
        self.camera = VideoCapture(0)
        self.camera.set(CAP_PROP_FRAME_WIDTH, 320)
        self.camera.set(CAP_PROP_FRAME_HEIGHT, 240)
        time.sleep(2)

        if not self.camera.isOpened():
            raise Exception("Could not open camera")

    def DetectQRCode(self, frame):
        """Detect and process QR Code in frame"""
        decoded_qrs = decode_qr(frame)
        imshow('QR Detection', frame)

        if decoded_qrs == None:
            print('decoded_qrs is None')
            raiseExceptions

        # Process detected QR code
        for qr_code in decoded_qrs:
            # Get the barcode's data and type
            data = qr_code.data.decode('utf-8')

            # Get QR code position and dimensions
            qr_rect = qr_code.rect
            qr_left = qr_rect.left
            qr_top = qr_rect.top
            qr_width = qr_rect.width
            qr_height = qr_rect.height

            # Calculate center coordinates
            qr_center_x = qr_left + (qr_width / 2)
            qr_center_y = qr_top + (qr_height / 2)

            # Calculate distance from frame center
            offset_x = qr_center_x - (frame.shape[1] / 2)
            offset_y = qr_center_y - (frame.shape[0] / 2)

            # Draw outline around detected QR code
            rectangle(frame,
                      (qr_left, qr_top),
                      (qr_left + qr_width, qr_top + qr_height),
                      (0, 255, 0),  # Green color
                      6)  # Line thickness

            imshow('QR Detection', frame)
            return data, offset_x, offset_y

    def run(self):
        """Continuously read QR codes"""
        while True:
            ret, frame = self.camera.read()
            if not ret:
                print("Failed to grab frame")
                continue

            try:
                data, offset_x, offset_y = self.DetectQRCode(frame)
                if data is not None:
                    print(f"Data: {data}")
                    print(f"X_offset: {offset_x:.1f}")
                    print(f"Y_offset: {offset_y:.1f}")

            except:
                pass

            if waitKey(1) & 0xFF == ord('q'):
                break

        self.cleanup()

    def cleanup(self):
        """Release camera and cleanup"""
        self.camera.release()
        destroyAllWindows()


def main():
    try:
        detector = QRCodeDetect()
        detector.run()

    except Exception as e:
        print(f"An error occurred: {str(e)}")


if __name__ == "__main__":
    main()